In [19]:
import glob
import os
import pickle
import matplotlib.pyplot as plt
from PIL import Image
import shutil

data_folder_path = '/home/osero/Downloads/mmpose-full/0001/'
img_folder = '/home/osero/Downloads/frame-full-c1080-s10/0001'
result_folder = 'result_hand_left/0001/'

data_files = [os.path.basename(file) for file in glob.glob(os.path.join(data_folder_path, '*')) if os.path.isfile(file)]

padding = 0
for file_path in data_files:
    user_name = os.path.splitext(file_path)[0]
    data_path = data_folder_path + file_path
    file = open(data_path, 'rb')

    # dump information to that file
    data = pickle.load(file)

    frame_count = data['hand_left']['left_lunate_bone'].shape[0]
    save_folder = os.path.join(result_folder, user_name)
    if os.path.exists(save_folder):
        shutil.rmtree(save_folder)

    os.makedirs(save_folder)
    for frame_number in range(0,frame_count):
        x_max = 0
        y_max = 0
        x_min = 1920
        y_min = 1080
        for finger_key, finger_value in data['hand_left'].items():
            cx = finger_value[frame_number][0]
            cy = finger_value[frame_number][1]
            if cx > x_max:
                x_max = cx
            if cx < x_min:
                x_min = cx
            if cy > y_max:
                y_max = cy
            if cy < y_min:
                y_min = cy
        img_path = "%s/%s/%s.jpg" % (img_folder, user_name, str(frame_number+1).zfill(5))
        result_img_path = "%s/%s/%s.jpg" % (result_folder, user_name, str(frame_number+1).zfill(5))
        img = Image.open(img_path)
        cropped_img = img.crop([x_min-padding, y_min-padding, x_max+padding, y_max+padding])
        cropped_img.save(result_img_path)  # Change this path as needed
        # plt.imshow(cropped_img)
        # plt.show()
    # close the file
    file.close()




FileNotFoundError: [Errno 2] No such file or directory: '/home/osero/Desktop/CMPE/dinov2/preprocess/result_hand_left/0001/User_2_003/00001.jpg'

In [1]:
import glob
import os
folder_path = '/home/osero/Downloads/frame-full-c1080-s10/0001/User_5_001'
files = [os.path.basename(file) for file in glob.glob(os.path.join(folder_path, '*')) if os.path.isfile(file)]
files

['00052.jpg',
 '00027.jpg',
 '00032.jpg',
 '00070.jpg',
 '00077.jpg',
 '00031.jpg',
 '00060.jpg',
 '00047.jpg',
 '00007.jpg',
 '00054.jpg',
 '00059.jpg',
 '00056.jpg',
 '00034.jpg',
 '00042.jpg',
 '00040.jpg',
 '00083.jpg',
 '00030.jpg',
 '00044.jpg',
 '00066.jpg',
 '00085.jpg',
 '00008.jpg',
 '00078.jpg',
 '00068.jpg',
 '00055.jpg',
 '00019.jpg',
 '00014.jpg',
 '00069.jpg',
 '00063.jpg',
 '00081.jpg',
 '00053.jpg',
 '00006.jpg',
 '00038.jpg',
 '00045.jpg',
 '00026.jpg',
 '00029.jpg',
 '00017.jpg',
 '00002.jpg',
 '00072.jpg',
 '00035.jpg',
 '00018.jpg',
 '00022.jpg',
 '00020.jpg',
 '00043.jpg',
 '00061.jpg',
 '00075.jpg',
 '00009.jpg',
 '00071.jpg',
 '00004.jpg',
 '00011.jpg',
 '00051.jpg',
 '00036.jpg',
 '00024.jpg',
 '00073.jpg',
 '00003.jpg',
 '00050.jpg',
 '00048.jpg',
 '00023.jpg',
 '00015.jpg',
 '00076.jpg',
 '00028.jpg',
 '00021.jpg',
 '00037.jpg',
 '00062.jpg',
 '00080.jpg',
 '00046.jpg',
 '00001.jpg',
 '00012.jpg',
 '00033.jpg',
 '00084.jpg',
 '00079.jpg',
 '00057.jpg',
 '0005

In [27]:
import shutil
import cv2
import mediapipe as mp

result_path = 'results_hand'

# Initializing the Model
min_detection_confidence = 0.5
mpHands = mp.solutions.hands
hands = mpHands.Hands(
    static_image_mode = True,
    model_complexity = 1,
    min_detection_confidence = min_detection_confidence,
    max_num_hands = 2)

def crop_hand_images (all_images):
    global counter_single, counter_none, counter_all
    # hands.reset()
    padding = 10
    for file_name, image in all_images:
        # Process the RGB image
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = hands.process(image)
        h, w, c = image.shape
        counter_all += 1
        # If hands are present in image(frame)
        if results.multi_hand_landmarks:
            for idx, handLMs in enumerate(results.multi_hand_landmarks):
                x_max = 0
                y_max = 0
                x_min = w
                y_min = h
                lmlist = []
                for lm in handLMs.landmark:
                    cx, cy = int(lm.x * w), int(lm.y * h)
                    if cx > x_max:
                        x_max = cx
                    if cx < x_min:
                        x_min = cx
                    if cy > y_max:
                        y_max = cy
                    if cy < y_min:
                        y_min = cy

                hand_img = image[y_min-padding:y_max+padding,x_min-padding:x_max+padding]
                cv2.imwrite(result_path + '/' + file_name + '_h' + str(idx) + '.jpg', cv2.cvtColor(hand_img, cv2.COLOR_RGB2BGR))

                # plt.imshow(Image.fromarray(image[y_min-padding:y_max+padding,x_min-padding:x_max+padding]))
                # plt.show()
                aaa = 5

            if len(results.multi_handedness) != 2:
                # plt.imshow(Image.fromarray(image))
                # plt.show()
                counter_single += 1

        else:
            counter_none += 1


libEGL warning: MESA-LOADER: failed to open iris: /usr/lib/dri/iris_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open iris: /usr/lib/dri/iris_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open iris: /usr/lib/dri/iris_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open iris: /usr/lib/dri/iris_dri.so: cannot open shared object file: 

W0000 00:00:1728689938.970816   98346 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1728689938.989209   98346 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [28]:
import cv2 
from matplotlib import pyplot as plt
from PIL import Image

frame_sample_frequency = 8

if os.path.exists(result_path):
    shutil.rmtree(result_path)

os.makedirs(result_path)

counter_single = 0
counter_none = 0
counter_all = 0

for file_path in files:
    # Open the video file
    video_path = folder_path + file_path
    video_capture = cv2.VideoCapture(video_path)

    # Get the total number of frames
    total_frames = int(video_capture.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_numbers = [i for i in range(0, total_frames, frame_sample_frequency)]
    all_images = []

    for frame_number in frame_numbers:
            
        # Set the video capture to requested frame number
        video_capture.set(cv2.CAP_PROP_POS_FRAMES, frame_number)

        # Read the frame
        success, img = video_capture.read()
        if success:
            frame_file_path = os.path.splitext(file_path)[0] + '_f' + str(frame_number)
            all_images.append((frame_file_path, img))
        else:
            print('Error while reading image from a video')
            print('File:', file_path)
            print('total_frames:', total_frames)
            print('frame_number:', frame_number)
    
    create_hand_images(all_images)

print('min_detection_confidence:', min_detection_confidence)
print('All:', counter_all)
print('Missed:', counter_none)
print('Single:', counter_single)

/home/osero/miniconda3/envs/dinov2/lib/python3.10/site-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


min_detection_confidence: 0.5
All: 7554
Missed: 291
Single: 1178


In [5]:

# import shutil
# import cv2
# import mediapipe as mp

# # Used to convert protobuf message
# # to a dictionary.
# from google.protobuf.json_format import MessageToDict

# result_path = 'results_hand'
# if os.path.exists(result_path):
#     shutil.rmtree(result_path)

# os.makedirs(result_path)
# # Initializing the Model
# min_detection_confidence = 0.5
# mpHands = mp.solutions.hands
# hands = mpHands.Hands(
#     static_image_mode=True,
#     model_complexity=1,
#     min_detection_confidence = min_detection_confidence,
#     max_num_hands=2)

# padding = 10
# counter_single = 0
# counter_none = 0
# for file_name, image in all_images:
#     # Process the RGB image
#     image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#     results = hands.process(image)
#     h, w, c = image.shape

#     # If hands are present in image(frame)
#     if results.multi_hand_landmarks:
#         for idx, handLMs in enumerate(results.multi_hand_landmarks):
#             x_max = 0
#             y_max = 0
#             x_min = w
#             y_min = h
#             lmlist = []
#             for lm in handLMs.landmark:
#                 cx, cy = int(lm.x * w), int(lm.y * h)
#                 if cx > x_max:
#                     x_max = cx
#                 if cx < x_min:
#                     x_min = cx
#                 if cy > y_max:
#                     y_max = cy
#                 if cy < y_min:
#                     y_min = cy
#                 # lmlist.append([cx,cy])
#                 # cv2.circle(image,(cx,cy),5,(255,0,255),cv2.FILLED)
#             hand_img = image[y_min-padding:y_max+padding,x_min-padding:x_max+padding]
#             cv2.imwrite(result_path + '/' + file_name + '_h' + str(idx) + '.jpg', hand_img)

#             # plt.imshow(Image.fromarray(image[y_min-padding:y_max+padding,x_min-padding:x_max+padding]))
#             # plt.show()
#             aaa = 5

#         # Both Hands are present in image(frame)
#         if len(results.multi_handedness) != 2:
#             # plt.imshow(Image.fromarray(image))
#             # plt.show()
#                 # Display 'Both Hands' on the image
#             counter_single += 1

#     else:
#         counter_none += 1
#     # # Display Video and when 'q' is entered, destroy the window
#     # cv2.imshow('Image', img)
# print('min_detection_confidence:', min_detection_confidence)
# print('All:', len(all_images))
# print('Missed:', counter_none)
# print('Single:', counter_single)
# bb = 5


# # import shutil
# # result_path = 'results_hand'
# # if os.path.exists(result_path):
# #     shutil.rmtree(result_path)

# # os.makedirs(result_path)

# # face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_alt.xml')
# # jdx = 0
# # for image in all_images:
# #     jdx += 1
# #     # Detect faces
# #     gray_img = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
# #     faces = face_cascade.detectMultiScale(gray_img, 1.2, minSize = [30,30])
# #     #plt.imshow(Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB)))
# #     #plt.show()
# #     for idx, (x, y, w, h) in enumerate(faces):
# #         padding = 10
# #         face_img = image[y - padding:y + h + padding, x - padding:x + w + padding]
# #         cropped_head_pil = Image.fromarray(cv2.cvtColor(face_img, cv2.COLOR_BGR2RGB))
# #         #plt.imshow(cropped_head_pil)
# #         cv2.imwrite(result_path + '/' + str(jdx) + 'face' + str(idx) + '.jpg', face_img)



libEGL warning: MESA-LOADER: failed to open iris: /usr/lib/dri/iris_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open iris: /usr/lib/dri/iris_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open iris: /usr/lib/dri/iris_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open iris: /usr/lib/dri/iris_dri.so: cannot open shared object file: 

error: OpenCV(4.10.0) /io/opencv/modules/imgproc/src/color.cpp:196: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'
